In [ ]:
!pip install pandas_ta --quiet

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow as tf
print(tf.keras.__version__)

In [ ]:
!pip install scikit-learn tensorflow pandas_ta


In [ ]:
!pip install --upgrade numpy pandas

In [3]:
!pip uninstall numpy -y


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [6]:
!pip install numpy==1.26.4

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 9.0 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 3.8.0 which is incompatible.
tensorflow-macos 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.4 which is incompatible.
tensorflow-macos 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.18.0 which is incompatible.


In [1]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 1.26.4
Pandas version: 2.2.3


In [ ]:
from tensorflow.keras import layers

In [3]:
import pandas as pd
import numpy as np
import pandas_ta as ta  # for MACD, SMA, etc.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
print(pd.__version__)
print(np.__version__)

pd.set_option('display.max_columns', 50)

2.18.0
2.2.3
1.26.4


In [4]:
# Load the merged CSV with Apple prices + sentiment.

csv_file = "merged_news_stock_data.csv"  # adjust if needed

df = pd.read_csv(csv_file, parse_dates=["Date"])
df.sort_values("Date", inplace=True)
df.dropna(subset=["Date"], inplace=True)
df.reset_index(drop=True, inplace=True)

print("Columns in dataset:", df.columns.tolist())
print("Row count:", len(df))
df.head()

Columns in dataset: ['Date', 'title', 'content', 'link', 'symbols', 'tags', 'sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'Close_Price', 'High_Price', 'Low_Price', 'Open_Price', 'Volume']
Row count: 26581


,Date,title,content,link,symbols,tags,sentiment_polarity,sentiment_neg,sentiment_neu,sentiment_pos,Close_Price,High_Price,Low_Price,Open_Price,Volume
0,2016-02-19,Payment Data Systems Announces Apple Pay Suppo...,"SAN ANTONIO, Feb. 19, 2016 (GLOBE NEWSWIRE) ...",https://www.globenewswire.com/news-release/201...,"AAPL.US, PYDS.US, USIO.US","AKIMBO, AKIMBO CARD, APPLE, APPLE PAY, FICENTI...",0.994,0.023,0.869,0.108,21.845985,22.009762,21.791394,21.836887,141496800
1,2017-10-05,New Research: Key Drivers of Growth for Micros...,"NEW YORK, Oct. 05, 2017 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/201...,"AAPL.US, AMZN.US, INTC.US, MSFT.US, QCOM.US, T...",FUNDAMENTAL MARKETS,0.997,0.008,0.925,0.067,36.384716,36.396424,36.070955,36.101392,85135200
2,2017-11-27,"Factors of Influence in 2018, Key Indicators a...","NEW YORK, Nov. 27, 2017 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/201...,"AAPL.US, CSCO.US, GD.US, HPE.US, NVDA.US, S.US",FUNDAMENTAL MARKETS,0.997,0.008,0.926,0.065,40.909874,41.142518,40.733629,41.135469,82867200
3,2017-11-30,"BioTelemetry, Inc. Enters Agreement to Provide...","MALVERN, Pa., Nov. 30, 2017 (GLOBE NEWSWIRE)...",https://www.globenewswire.com/news-release/201...,"AAPL.US, BEAT.US","BIOTELEMETRY, INC, NASDAQ:BEAT",0.989,0.021,0.804,0.174,40.383492,40.451638,39.582166,40.049799,166108800
4,2018-01-31,Investor Expectations to Drive Momentum within...,"NEW YORK, Jan. 31, 2018 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/201...,"AAPL.US, AVHI.US, FARM.US, GM.US, SGRY.US",FUNDAMENTAL MARKETS,0.995,0.009,0.937,0.054,39.344833,39.582178,39.126292,39.213238,129915600


In [5]:
print("Initial columns:", df.columns)

Initial columns: Index(['Date', 'title', 'content', 'link', 'symbols', 'tags',
       'sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos',
       'Close_Price', 'High_Price', 'Low_Price', 'Open_Price', 'Volume'],
      dtype='object')


3) Feature Engineering & Label Creation

In [ ]:
# We'll assume the CSV has a 'Close' column for the daily close.
# If you also have 'Open', 'High', 'Low', 'Volume', feel free to use them.

# RSI (14-day)
df["RSI"] = ta.rsi(df["Close_Price"], length=14)

# MACD
macd_data = ta.macd(df["Close_Price"])  # yields MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9
df["MACD"] = macd_data["MACD_12_26_9"]
df["MACD_signal"] = macd_data["MACDs_12_26_9"]  # the signal line
# optional: df["MACD_hist"] = macd_data["MACDh_12_26_9"]

# Moving Average (50-day)
df["SMA_50"] = ta.sma(df["Close_Price"], length=50)

# Drop rows that turned NaN due to rolling calculations
df.dropna(subset=["RSI", "MACD", "MACD_signal", "SMA_50"], inplace=True)
df.reset_index(drop=True, inplace=True)

print("After adding RSI, MACD, SMA_50:")
df.head()

After adding RSI, MACD, SMA_50:


,Date,title,content,link,symbols,tags,sentiment_polarity,sentiment_neg,sentiment_neu,sentiment_pos,Close_Price,High_Price,Low_Price,Open_Price,Volume,RSI,MACD,MACD_signal,SMA_50
0,2020-03-23,3 ETFs To Short The Dow,"With last Friday's 4.55% slide, the Dow Jones ...",https://finance.yahoo.com/news/3-etfs-short-do...,"AAPL.US, BA.US, CSCO.US, DIA.US, DJI.INDX, DOG...","DOW JONES INDUSTRIAL AVERAGE, NYSE",0.994,0.020,0.869,0.111,54.449890,55.452156,51.595986,55.350231,336752800,76.014186,1.224731,0.487131,44.301828
1,2020-03-24,Are Sporting Goods Essential in a Pandemic?,(Bloomberg Opinion) -- You really can’t blame ...,https://finance.yahoo.com/news/sporting-goods-...,"AAPL.BA, AAPL.US, AAPL34.SA, FRAS.LSE, KGF.LSE...","APPLIANCE STORES, BLOOMBERG OPINION, EVANS CYC...",0.940,0.076,0.830,0.094,59.912601,60.109171,56.859699,57.359617,287531200,81.156431,2.221344,0.833974,45.063161
2,2020-03-27,Olympics delay deals setback to Samsung's plan...,By Hyunjoo Jin\n\nSEOUL (Reuters) - For Samsun...,https://finance.yahoo.com/news/olympics-delay-...,"AAPL.US, DCMYY.US, SSNLF.US","APPLE INC, JAPAN, SAMSUNG ELECTRONICS CO LTD",0.980,0.041,0.881,0.078,60.121304,62.094282,59.953854,61.337124,204216800,81.321191,2.993500,1.265879,45.537893
3,2020-03-27,Coronavirus Is Expediting iPhone Makers' Plans...,"(Bloomberg) -- Wistron Corp., one of Apple’s m...",https://finance.yahoo.com/news/coronavirus-exp...,"1810.HK, 2317.TW, 2356.TW, 2477.TW, 3231.TW, 4...","APPLE, BLOOMBERG, CHINA, HON HAI PRECISION IND...",0.979,0.016,0.921,0.062,60.121304,62.094282,59.953854,61.337124,204216800,81.321191,3.564351,1.725573,45.922121
4,2020-03-30,Buying What the Fed Buys,"On Monday, March 23, the Federal Reserve annou...",https://finance.yahoo.com/news/buying-fed-buys...,"AAPL.US, AMZN.US, FB.US, GOOGL.US, IGSB.US, JP...","BOND ETF, CORPORATE BOND ETF, CORPORATE BONDS,...",0.992,0.057,0.846,0.098,61.837048,62.009351,60.524153,60.849346,167976400,82.758523,4.107849,2.202029,46.351192


In [ ]:
# We might add other indicators similarly, e.g. RSI
# df["RSI"] = ta.rsi(df["Close"], length=14)

In [7]:
df["future_close"] = df["Close_Price"].shift(-1)
df["target_up"] = (df["future_close"] > df["Close_Price"]).astype(int)
# Drop last row which has no future_close
df.dropna(subset=["future_close"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Date,title,content,link,symbols,tags,sentiment_polarity,sentiment_neg,sentiment_neu,sentiment_pos,Close_Price,High_Price,Low_Price,Open_Price,Volume,RSI,MACD,MACD_signal,SMA_50,future_close,target_up
0,2020-03-23,3 ETFs To Short The Dow,"With last Friday's 4.55% slide, the Dow Jones ...",https://finance.yahoo.com/news/3-etfs-short-do...,"AAPL.US, BA.US, CSCO.US, DIA.US, DJI.INDX, DOG...","DOW JONES INDUSTRIAL AVERAGE, NYSE",0.994,0.020,0.869,0.111,54.449890,55.452156,51.595986,55.350231,336752800,76.014186,1.224731,0.487131,44.301828,59.912601,1
1,2020-03-24,Are Sporting Goods Essential in a Pandemic?,(Bloomberg Opinion) -- You really can’t blame ...,https://finance.yahoo.com/news/sporting-goods-...,"AAPL.BA, AAPL.US, AAPL34.SA, FRAS.LSE, KGF.LSE...","APPLIANCE STORES, BLOOMBERG OPINION, EVANS CYC...",0.940,0.076,0.830,0.094,59.912601,60.109171,56.859699,57.359617,287531200,81.156431,2.221344,0.833974,45.063161,60.121304,1
2,2020-03-27,Olympics delay deals setback to Samsung's plan...,By Hyunjoo Jin\n\nSEOUL (Reuters) - For Samsun...,https://finance.yahoo.com/news/olympics-delay-...,"AAPL.US, DCMYY.US, SSNLF.US","APPLE INC, JAPAN, SAMSUNG ELECTRONICS CO LTD",0.980,0.041,0.881,0.078,60.121304,62.094282,59.953854,61.337124,204216800,81.321191,2.993500,1.265879,45.537893,60.121304,0
3,2020-03-27,Coronavirus Is Expediting iPhone Makers' Plans...,"(Bloomberg) -- Wistron Corp., one of Apple’s m...",https://finance.yahoo.com/news/coronavirus-exp...,"1810.HK, 2317.TW, 2356.TW, 2477.TW, 3231.TW, 4...","APPLE, BLOOMBERG, CHINA, HON HAI PRECISION IND...",0.979,0.016,0.921,0.062,60.121304,62.094282,59.953854,61.337124,204216800,81.321191,3.564351,1.725573,45.922121,61.837048,1
4,2020-03-30,Buying What the Fed Buys,"On Monday, March 23, the Federal Reserve annou...",https://finance.yahoo.com/news/buying-fed-buys...,"AAPL.US, AMZN.US, FB.US, GOOGL.US, IGSB.US, JP...","BOND ETF, CORPORATE BOND ETF, CORPORATE BONDS,...",0.992,0.057,0.846,0.098,61.837048,62.009351,60.524153,60.849346,167976400,82.758523,4.107849,2.202029,46.351192,61.837048,0


4) Select Feature Columns

In [8]:
feature_cols = [
    "Close_Price", 
    "Volume",
    "RSI", 
    "MACD", 
    "MACD_signal", 
    "SMA_50",
    "sentiment_polarity",
    "sentiment_neg",
    "sentiment_neu",
    "sentiment_pos"
]

df_model = df[["Date"] + feature_cols + ["target_up"]].copy()
df_model.dropna(inplace=True)
print("df_model columns:", df_model.columns.tolist())
df_model.head()

df_model columns: ['Date', 'Close_Price', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'SMA_50', 'sentiment_polarity', 'sentiment_neg', 'sentiment_neu', 'sentiment_pos', 'target_up']


,Date,Close_Price,Volume,RSI,MACD,MACD_signal,SMA_50,sentiment_polarity,sentiment_neg,sentiment_neu,sentiment_pos,target_up
0,2020-03-23,54.449890,336752800,76.014186,1.224731,0.487131,44.301828,0.994,0.020,0.869,0.111,1
1,2020-03-24,59.912601,287531200,81.156431,2.221344,0.833974,45.063161,0.940,0.076,0.830,0.094,1
2,2020-03-27,60.121304,204216800,81.321191,2.993500,1.265879,45.537893,0.980,0.041,0.881,0.078,0
3,2020-03-27,60.121304,204216800,81.321191,3.564351,1.725573,45.922121,0.979,0.016,0.921,0.062,1
4,2020-03-30,61.837048,167976400,82.758523,4.107849,2.202029,46.351192,0.992,0.057,0.846,0.098,0


In [9]:
# 1) Scale features (so RSI, Volume, Sentiment are on similar numeric scales)
scaler = StandardScaler()

df_model_scaled = df_model.copy()
df_model_scaled[feature_cols] = scaler.fit_transform(df_model_scaled[feature_cols])

# 2) Convert to sequences
lookback = 5

vals = df_model_scaled[feature_cols].values  # shape (N, num_features)
targets = df_model_scaled["target_up"].values

X, y = [], []
for i in range(len(vals) - lookback):
    seq_x = vals[i : i+lookback]
    label = targets[i + lookback]
    X.append(seq_x)
    y.append(label)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape, "y shape:", y.shape)
# e.g. X shape: (samples, 5, #features), y shape: (samples,)

X shape: (26513, 5, 10) y shape: (26513,)


In [10]:
train_size = int(0.8 * len(X))
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

print("Train size:", X_train.shape[0], "Test size:", X_test.shape[0])


Train size: 21210 Test size: 5303


In [11]:
model = keras.Sequential()
model.add(layers.Input(shape=(lookback, len(feature_cols))))
model.add(layers.LSTM(64, return_sequences=False))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,265 (75.25 KB)

 Trainable params: 19,265 (75.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
epochs = 10
batch_size = 32

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

Epoch 1/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9329 - loss: 0.2138 - val_accuracy: 0.9804 - val_loss: 0.1014
Epoch 2/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9777 - loss: 0.1014 - val_accuracy: 0.9804 - val_loss: 0.0989
Epoch 3/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9795 - loss: 0.0946 - val_accuracy: 0.9804 - val_loss: 0.0978
Epoch 4/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9777 - loss: 0.0976 - val_accuracy: 0.9804 - val_loss: 0.0953
Epoch 5/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9788 - loss: 0.0948 - val_accuracy: 0.9804 - val_loss: 0.0969
Epoch 6/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9786 - loss: 0.0953 - val_accuracy: 0.9804 - val_loss: 0.0960
Epoch 7/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9806 - loss: 0.0884 - val_accuracy: 0.9804 - val_loss: 0.0958
Epoch 8/10
531/531 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9817 - loss: 0.0845 - val_accuracy: 0.

In [18]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)
print(classification_report(y_test, y_pred))

166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step
Test Accuracy: 0.9732227041297379
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      5161
           1       0.00      0.00      0.00       142

    accuracy                           0.97      5303
   macro avg       0.49      0.50      0.49      5303
weighted avg       0.95      0.97      0.96      5303



/Users/shobians./Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shobians./Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shobians./Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
